### importer les bibliotheque 

In [23]:
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error

#### Connecter a mon BDD

In [24]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv

# Charger le fichier .env
load_dotenv()

# Récupérer les infos de connexion depuis l'env
host = os.getenv("MYSQL_HOST")
database = os.getenv("MYSQL_DATABASE")
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")

try:
    connection = mysql.connector.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        charset="utf8mb4",
        collation="utf8mb4_general_ci"
    )
    if connection.is_connected():
        db_Info = connection.server_info
        print("Vous êtes connecté à la version MySQL Server ", db_Info)

        cursor = connection.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("Vous êtes connecté à la base de données: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)


Vous êtes connecté à la version MySQL Server  11.4.2-MariaDB
Vous êtes connecté à la base de données:  ('bibliotheque',)


### Requettes sur la base de données 

In [25]:
req_1 = "SELECT * FROM livres;"
cursor.execute(req_1)
result_1 = cursor.fetchall()
result_1
df_livres = pd.DataFrame(result_1, columns=cursor.column_names)
df_livres.head()

,id_livre,titre,annee_publication,id_auteur
0,1,Les Misérables,1862,1
1,2,Notre-Dame de Paris,1831,1
2,3,Harry Potter 1,1997,2
3,4,Harry Potter 2,1998,2
4,5,Le Seigneur des Anneaux,1954,3


a)  Quels sont les livres empruntés cette semaine ? 

In [26]:
req_a = """
SELECT l.titre, e.date_emprunt, em.nom, em.prenom
FROM emprunts e
JOIN livres l ON e.id_livre = l.id_livre
JOIN emprunteurs em ON e.id_emprunteur = em.id_emprunteur
WHERE e.date_emprunt >= DATE_SUB(CURDATE(), INTERVAL 7 DAY);
"""

cursor.execute(req_a)
result_a = cursor.fetchall()
result_a


for row in result_a:
    print("Titre du livre:", row[0])
    print("Date d'emprunt:", row[1])
    print("Nom de l'emprunteur:", row[2])
    print("Prénom de l'emprunteur:", row[3])
    print("-----------------------------")

Titre du livre: Harry Potter 1
Date d'emprunt: 2025-11-18
Nom de l'emprunteur: Dupont
Prénom de l'emprunteur: Alice
-----------------------------
Titre du livre: Les Misérables
Date d'emprunt: 2025-11-17
Nom de l'emprunteur: Martin
Prénom de l'emprunteur: Leo
-----------------------------
Titre du livre: Harry Potter 2
Date d'emprunt: 2025-11-20
Nom de l'emprunteur: Bernard
Prénom de l'emprunteur: Sofia
-----------------------------


b) Quel·le auteur·e a publié le plus de livres ? 

In [27]:
req_b = """
SELECT a.nom, a.prenom, COUNT(*) AS total_livres
FROM livres l
JOIN auteurs a ON l.id_auteur = a.id_auteur
GROUP BY a.id_auteur
ORDER BY total_livres DESC
LIMIT 1;

"""

cursor.execute(req_b)
result_b = cursor.fetchone()
print("Auteur avec le plus de livres dans la bibliothèque:")
print("Nom:", result_b[0])
print("Prénom:", result_b[1])
print("Nombre total de livres:", result_b[2])


Auteur avec le plus de livres dans la bibliothèque:
Nom: Hugo
Prénom: Victor
Nombre total de livres: 2


c) Quels emprunteur·euses ont emprunté le plus de livres sur une période donnée ? 

In [28]:
result_c ="""
SELECT em.nom, em.prenom, COUNT(*) AS total_emprunts
FROM emprunts e
JOIN emprunteurs em ON e.id_emprunteur = em.id_emprunteur
WHERE e.date_emprunt BETWEEN '2025-11-01' AND '2025-11-30'
GROUP BY em.id_emprunteur
ORDER BY total_emprunts DESC;
"""

cursor.execute(result_c)
result_c = cursor.fetchall()
result_c
for row in result_c:
    print("Nom de l'emprunteur:", row[0])
    print("Prénom de l'emprunteur:", row[1])
    print("Nombre total d'emprunts en novembre 2025:", row[2])
    print("-----------------------------")

Nom de l'emprunteur: Dupont
Prénom de l'emprunteur: Alice
Nombre total d'emprunts en novembre 2025: 2
-----------------------------
Nom de l'emprunteur: Martin
Prénom de l'emprunteur: Leo
Nombre total d'emprunts en novembre 2025: 1
-----------------------------
Nom de l'emprunteur: Bernard
Prénom de l'emprunteur: Sofia
Nombre total d'emprunts en novembre 2025: 1
-----------------------------


Vue : Liste des emprunts en cours

In [29]:
req_d = """

CREATE VIEW vue_emprunts_en_cours AS
SELECT e.id_emprunt, l.titre, em.nom, em.prenom, e.date_emprunt
FROM emprunts e
JOIN livres l ON e.id_livre = l.id_livre
JOIN emprunteurs em ON e.id_emprunteur = em.id_emprunteur
WHERE e.date_retour IS NULL;

"""

cursor.execute(req_d)
result_d = cursor.fetchall()
result_d

[]

Vue : Nombre de livres par auteur

In [30]:
req_e = """

CREATE VIEW vue_nb_livres_par_auteur AS
SELECT a.id_auteur, a.nom, a.prenom, COUNT(l.id_livre) AS total_livres
FROM auteurs a
LEFT JOIN livres l ON a.id_auteur = l.id_auteur
GROUP BY a.id_auteur;

"""
cursor.execute(req_e)
result_e = cursor.fetchall()
result_e

[]

Vue : Statistiques d’emprunts par utilisateur

In [31]:
req_f=  """   
CREATE VIEW vue_stats_emprunteurs AS
SELECT em.id_emprunteur, em.nom, em.prenom, COUNT(e.id_emprunt) AS total_emprunts
FROM emprunteurs em
LEFT JOIN emprunts e ON em.id_emprunteur = e.id_emprunteur
GROUP BY em.id_emprunteur;

"""

cursor.execute(req_f)
result_f = cursor.fetchall()
result_f

[]